# Layout Script v1 #

- muda apenas layer com valores quantitativos apresentados por intervalos

In [ ]:
from gesri.lyt.pmap import layoutv1

mxd        = r'D:\mystuff\gowe_layouts\zzgowe\zzgowe.aprx'

mapfiles = [{
    "maps"    : r'D:\mystuff\gowe_layouts\lyttblmain\tvl1jenks.xlsx',
    "geodata" : r"D:\mystuff\gowe_layouts\lyt1shp_jenks",
    "outmaps" : r'D:\mystuff\gowe_layouts\lyt1_jenks'
}]
""", {
    "maps"    : r'D:\mystuff\gowe_layouts\tv_maps_l1_jenks.xlsx',
    "geodata" : r"D:\mystuff\gowe_layouts\lytshp_jenks",
    "outmaps" : r'D:\mystuff\gowe_layouts\lyt1_jenks'
}, {
    "maps"    : r'D:\mystuff\gowe_layouts\tv_maps_l1_quantile.xlsx',
    "geodata" : r"D:\mystuff\gowe_layouts\lytshp_qtl",
    "outmaps" : r'D:\mystuff\gowe_layouts\lyt1_qtl'
}]"""

lyt_template = 'map_template'
map_template = 'map_layout'
lyr_int      = 'fregtv'

for d in mapfiles:
    layoutv1(
        mxd, d["geodata"], d["maps"], lyt_template,
        map_template, lyr_int, d['outmaps']
    )

# Layout Script v2 #

- muda layer com valores quantitativos apresentados por intervalos e outras layers sem valores associados, apenas com simbologia (mudança source);

In [ ]:
import arcpy, pprint
from glass.dct import tbl_to_obj
from glass.pys.oss import lst_ff
import os

mxd        = r'D:\MEOCloud\gigscoimbra\zzproj\zzproj.aprx'
geodata    = r'D:\gwork\gigs_cmcprj\layouts_shp'
maps_table = r'D:\gwork\gigs_cmcprj\indicators\restantes_maps.xlsx'

other_layers_change = r'D:\gwork\gigs_cmcprj\Lista_mapas_09102020.xlsx'
other_layers_sheet  = 'restantes_other_layers'

other_layer_folder = r'D:\gwork\gigs_cmcprj\servicos\pnt_layout'

lyt_template = 'mortalidade'
map_template = 'map_layout'
lyr_int = 'freg_cmb_data'

out_maps = r'D:\gwork\gigs_cmcprj\layouts_v2'

In [ ]:
maps_todo = tbl_to_obj(maps_table)

maps_attr = list(maps_todo.columns.values)

lyr_change_by_map = tbl_to_obj(other_layers_change, sheet=other_layers_sheet)

In [ ]:
aprx = arcpy.mp.ArcGISProject(mxd)

# Get map
mapobj = aprx.listMaps(map_template)[0]
lyr = mapobj.listLayers(lyr_int)[0]

for i, r in maps_todo.iterrows():
    current_dict = lyr.connectionProperties

    replace_dict = {
        'connection_info' : {'database' : geodata},
        'dataset' : '{}.shp'.format(r.slug),
        'workspace_factory' : 'Shape File'
    }

    lyr.updateConnectionProperties(current_dict, replace_dict)
    
    # Update sources of other layers
    lchange = lyr_change_by_map[lyr_change_by_map.slug == r.slug]
    
    for _i, _r in lchange.iterrows():
        
        __lyr = mapobj.listLayers(_r.other_lyr)[0]
        c_d   = __lyr.connectionProperties
        
        rpdict = {
            'connection_info' : {'database' : other_layer_folder},
            'dataset' : _r.source_shp,
            'workspace_factory' : 'Shape File'
        }
        
        __lyr.updateConnectionProperties(c_d, rpdict)
    
    # Get Layout
    lyt = aprx.listLayouts(lyt_template)[0]
    
    # List elements
    elm = lyt.listElements("TEXT_ELEMENT")
    
    # Replace elements
    for e in elm:
        if e.name in maps_attr:
            if type(r[e.name]) == float:
                e.text = str(r[e.name]).replace('.', ',')
            else:
                e.text = str(r[e.name])

    lyt.exportToJPEG(os.path.join(
        out_maps, '{}.jpg'.format(r.slug)), resolution=500)

    aprx.saveACopy(os.path.join(out_maps, '{}.aprx'.format(r.slug)))

In [ ]:
lyr_change_by_map

# Layout Script v3 #

- alteracao de todos os dataframes (com excepção);

In [ ]:
import arcpy, pprint
from glass.dct import tbl_to_obj
from glass.pys.oss import lst_ff
import os

mxd        = r'D:\MEOCloud\gigscoimbra\cstemplate\cstemplate.aprx'
geodata    = r'D:\gwork\gigs_cmcprj\layouts_shp'
maps_table = r'D:\gwork\gigs_cmcprj\indicators\cs_maps.xlsx'

lyt_template = 'Layout'
map_template = 'map_layout'
lyr_int = 'freg_cmb_cs'

excepcao = ['Map']

out_maps = r'D:\gwork\gigs_cmcprj\layouts_v2'

In [ ]:
maps_todo = tbl_to_obj(maps_table)

maps_attr = list(maps_todo.columns.values)

In [ ]:
aprx = arcpy.mp.ArcGISProject(mxd)

# Get maps
mapobjs = aprx.listMaps()

mapobj = [m for m in mapobjs if m.name not in excepcao]

In [ ]:
lyrs = [m.listLayers(lyr_int)[0] for m in mapobj]

In [ ]:
print(len(lyrs))

In [ ]:
for i, r in maps_todo.iterrows():
    for lyr in lyrs:
        current_dict = lyr.connectionProperties

        replace_dict = {
            'connection_info' : {'database' : geodata},
            'dataset' : '{}.shp'.format(r.slug),
            'workspace_factory' : 'Shape File'
        }

        lyr.updateConnectionProperties(current_dict, replace_dict)
    
    # Get Layout
    lyt = aprx.listLayouts(lyt_template)[0]
    
    # List elements
    elm = lyt.listElements("TEXT_ELEMENT")
    
    # Replace elements
    for e in elm:
        if e.name in maps_attr:
            if type(r[e.name]) == float:
                e.text = str(r[e.name]).replace('.', ',')
            else:
                e.text = str(r[e.name])

    lyt.exportToJPEG(os.path.join(
        out_maps, '{}.jpg'.format(r.slug)), resolution=500)

    aprx.saveACopy(os.path.join(out_maps, '{}.aprx'.format(r.slug)))

In [ ]:
print(mapobj[0].name)

In [ ]:
print(len(mapobj))

## Layout v1 More than one map for layout

In [ ]:
mxd  = r'D:\mystuff\gowe_layouts\zzgowe\zzgowe.aprx'

maps = r'D:\mystuff\gowe_layouts\lyttblmain\tvl2jenks.xlsx'

shps = r'D:\mystuff\gowe_layouts\lyt2shp_jenks'

outmaps = r'D:\mystuff\gowe_layouts\lyt2_jenks'

map_template = {
    'map_duplo1' : {'col' : 'slug1', 'lyr' : 'fregtv'},
    'map_duplo2' : {'col' : 'slug2', 'lyr' : 'fregtv'}
}

lyt_template = 'map_duplo'

In [ ]:
import arcpy
import os
from glass.ng.rd import tbl_to_obj

In [ ]:
mapstodo = tbl_to_obj(maps)

In [ ]:
mapsattr = list(mapstodo.columns.values)

aprx = arcpy.mp.ArcGISProject(mxd)

In [ ]:
# Get template maps obj
# Get layer objs

for m in map_template:
    map_template[m]['mapobj'] = aprx.listMaps(m)[0]
    
    map_template[m]['lyrobj'] = map_template[m]['mapobj'].listLayers(
        map_template[m]['lyr'])[0]

In [ ]:
# Produce maps
for i, mp in mapstodo.iterrows():
    # Change all maps in map_template
    for mt in map_template:
        cdict = map_template[mt]['lyrobj'].connectionProperties
        
        rpdict = {
            'connection_info' : {'database' : shps},
            'dataset' : f'{mp[map_template[mt]["col"]]}.shp',
            'workspace_factory' : 'Shape File'
        }
        
        map_template[mt]['lyrobj'].updateConnectionProperties(cdict, rpdict)
        
    # Get layout
    lyt = aprx.listLayouts(lyt_template)[0]
    
    # List text elements
    elm = lyt.listElements("TEXT_ELEMENT")
    
    # Change text elements
    for e in elm:
        if e.name in mapsattr:
            if type(mp[e.name]) == float:
                e.text = str(mp[e.name]).replace('.', ',')
            
            else:
                e.text = str(mp[e.name])
    
    # Save new layout
    refname = "_".join([
        mp[map_template[_mt]['col']] for _mt in map_template
    ])
    lyt.exportToJPEG(os.path.join(
        outmaps, f'{refname}.jpg'
    ), resolution=500)
    
    aprx.saveACopy(os.path.join(outmaps, f'{refname}.aprx'))

In [ ]:
mapstodo